In [2]:

import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

e:\Diploma\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [3]:
# Load dataset
dataframe_path = "E:\\Diploma\\model_training\\datasets\\full_dataset.csv"

df = pd.read_csv(dataframe_path)

len(df)

4760963

In [4]:
# Load encoding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name).to(device)

In [ ]:
# Convert prompts to a Python list (if not already)
prompts_list = df['prompt'].astype(str).tolist()

# We will encode in batches to avoid running out of memory on large datasets
batch_size = 1048
embeddings = []

for start_idx in range(0, len(prompts_list), batch_size):
    batch_prompts = prompts_list[start_idx:start_idx + batch_size]
    batch_embeddings = model.encode(batch_prompts, 
                                    batch_size=batch_size, 
                                    show_progress_bar=True, 
                                    convert_to_numpy=True, 
                                    device=device)
    embeddings.append(batch_embeddings)
    
embeddings = np.vstack(embeddings)

print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
np.save("prompt_embeddings.npy", embeddings)
print("Embeddings saved to 'prompt_embeddings.npy'")